In [1]:
# Load the autoreload extension
%load_ext autoreload

# Set autoreload to automatically reload all modules
%autoreload 2

In [2]:
import sys
from pathlib import Path

# Add the src directory to sys.path
sys.path.append(str(Path().resolve() / "src"))

# Import util
import util

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Straakh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
import torch
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np

In [4]:
DATA_PATH = "../data/truthseeker.csv"
df = pd.read_csv(DATA_PATH)
# Comment on on actual execution. For development, this small sample should run quickly on the CPU
df = df.sample(frac=0.01,  random_state=27)
df.drop(columns=["Unnamed: 0"], inplace=True)
df = df[~df["5_label_majority_answer"].isin(["NO MAJORITY", "Unrelated"])]

clean_cols = ["statement", "tweet"]
df[clean_cols] = df[clean_cols].applymap(util.clean_text)

print(df.shape)
df.head(2)

(1123, 8)


C:\Users\Straakh\AppData\Local\Temp\ipykernel_4908\2912335117.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[clean_cols] = df[clean_cols].applymap(util.clean_text)


,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer
58262,April Hunt,Unlike marijuana medical cannabis oil cannot g...,True,1.0,"medical canabis, cannot, high",You cannot open a medical cannabis dispensary...,Agree,Agree
40146,Louis Jacobson,There are more words in the IRS code than the...,True,1.0,"More words, IRS code, Bible",There are more words in the IRS code than the...,Mostly Disagree,Disagree


In [5]:
aggregated_features = []
for statement, group in df.groupby("statement"):
    aggregated_sentiment, aggregated_embedding = util.extract_features(group["tweet"].tolist())
    aggregated_features.append([aggregated_sentiment, aggregated_embedding])

# Convert aggregated features into a new DataFrame
aggregated_df = pd.DataFrame(aggregated_features, columns=["Avg_Sentiment", "Avg_Embedding"])

# Flatten the embeddings into columns (assuming the embedding size is 768)
embedding_cols = [f"Embed_{i}" for i in range(768)]
embedding_df = pd.DataFrame(np.vstack(aggregated_df["Avg_Embedding"].to_numpy()), columns=embedding_cols)

In [6]:
# Concatenate the sentiment and embedding features
X = pd.concat([aggregated_df[["Avg_Sentiment"]], embedding_df], axis=1)

# Step 2: Define the true/false labels for statements
y = df.groupby("statement")['target'].first()

# Step 3: Train a classifier to predict the truth of a statement based on tweet features
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

# Use logistic regression for simplicity
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

# Step 4: Evaluate the model
y_pred = classifier.predict(X_test)

# Step 5: Output results
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Example of predicting a new statement truth based on tweets
new_tweets = [
    "This is a great article and provides true information.",
    "Politico's coverage of this is incorrect and false."
]
new_sentiment, new_embedding = util.extract_features(new_tweets)
X_new = np.concatenate([np.array([[new_sentiment]]), new_embedding.reshape(1, -1)], axis=1)  # Prepare the feature vector

y_new_pred = classifier.predict(X_new)

for i, label in enumerate(y_new_pred):
    print(f"Statement: '{new_tweets[i]}' - Predicted Truth: {'True' if label == 1 else 'False'}")

Classification Report:
              precision    recall  f1-score   support

       False       0.70      0.56      0.62        41
        True       0.66      0.78      0.71        45

    accuracy                           0.67        86
   macro avg       0.68      0.67      0.67        86
weighted avg       0.68      0.67      0.67        86

Statement: 'This is a great article and provides true information.' - Predicted Truth: False


c:\Users\Straakh\anaconda3\envs\truth\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
